In [1]:
import time
import random

def random_k_sat(n,m,k):
    # n variables, m clauses, k literals per clause
    clauses = []
    for _ in range(m):
        ret = []
        for _ in range(k):
            v = random.randrange(n) + 1
            polarity = random.randint(0, 1)
            ret.append( (2*polarity-1)*v ) 
        clauses.append(ret)
    return clauses, "&".join(
        "(" + "|".join(
            f"x{idx}" if idx>0 else "~" + f"x{-idx}" for 
            idx in cl
        ) + ")" for cl in clauses) 

def time_op(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        val = func(*args, **kwargs)
        dt_s = (time.time() - start)
        print(f"[{dt_s*1000:010.4f} ms / {dt_s:04.4f} s / {dt_s/60:02.4f} min] {func.__name__}")
        return val
    return wrapper

n,m,k = 10, 15, 3
cnf, formula = random_k_sat(n,m,k)
vars = [f"x{idx+1}" for idx in range(n)]

In [4]:
import importance_measures.sharpSAT as ssat
import importance_measures.bdds as bdd

solver = ssat.SharpSAT(decot=0.0002)
satcount_sharpsat = time_op(solver.satcount)(cnf)
print(f"SharpSAT-TD satcount = {satcount_sharpsat}")

with bdd.Buddy(vars) as model:
    f = time_op(model.node)(formula)
    satcount_bdd = time_op(lambda : f.satcount)()
    print(f"BDD satcount = {satcount_bdd}")

[00004.2145 ms / 0.0042 s / 0.0001 min] satcount
SharpSAT-TD satcount = 103
[00000.3424 ms / 0.0003 s / 0.0000 min] node
[00000.1590 ms / 0.0002 s / 0.0000 min] <lambda>
BDD satcount = 103.0


In [5]:
import importance_measures.bdds as bdd 
with bdd.Buddy(vars) as model:
    x = "x1"
    f = (time_op)(model.node)(formula)
    print(f"bdd size = {f.nodecount} with satcount = {f.satcount}")
    # f.dump("f.pdf")
    blame, ub = time_op(bdd.blame)(f, x, rho=lambda x: 1/(x+1), cutoff=0.1, debug=True)
    infl = time_op(bdd.influence)(f, x)
    # g = time_op(bdd.omega)(f)
    print(f"influence {infl:.5f}, {blame:.5f} <= blame <= {ub:.5f}, blame diff {ub-blame:.5f}")

[00000.3436 ms / 0.0003 s / 0.0000 min] node
bdd size = 43 with satcount = 103.0
=== COMPUTING BLAME for x1 in BDD with size 43 ===
k 0 size g 22 size ell 22 d result 0.1152 max increase possible 0.4424
k 1 size g 69 size ell 66 d result 0.1050 max increase possible 0.2249
k 2 size g 74 size ell 77 d result 0.0980 max increase possible 0.0952
stopped earlier because cannot improve above cutoff.
current value: 0.3182, can be increased by 0.0952 <= 0.1000.
=== DONE ===
[00004.2369 ms / 0.0042 s / 0.0001 min] blame
[00000.0181 ms / 0.0000 s / 0.0000 min] influence
influence 0.11523, 0.31820 <= blame <= 0.41341, blame diff 0.09521


In [6]:
import importance_measures.sharpSAT as ssat
f = time_op(ssat.Formula.parse)(formula)
x = "x1"
solver = ssat.SharpSAT(decot=1)
infl = (time_op)(ssat.influence)(f, x, solver)
blame, ub = time_op(ssat.blame)(f, x, solver, rho=lambda x: 1/(x+1), cutoff=0.1, debug=True)
print(f"influence {infl:.5f}, {blame:.5f} <= blame <= {ub:.5f}, blame diff {ub-blame:.5f}")

[00000.2847 ms / 0.0003 s / 0.0000 min] parse
[01008.8382 ms / 1.0088 s / 0.0168 min] influence
=== COMPUTING BLAME for x1 in Formula with size 428 ===
k 0 size g 1735 size ell 1744 d result 0.1152 max increase possible 0.4424
k 1 size g 17395 size ell 19140 d result 0.1050 max increase possible 0.2249
k 2 size g 158191 size ell 175596 d result 0.0980 max increase possible 0.0952
stopped earlier because cannot improve above cutoff.
current value: 0.3182, can be increased by 0.0952 <= 0.1000.
=== DONE ===
[06544.3819 ms / 6.5444 s / 0.1091 min] blame
influence 0.11523, 0.31820 <= blame <= 0.41341, blame diff 0.09521
